In [1]:
import pandas as pd
import datetime as dt
import pickle
import io
import os
import matplotlib.pyplot as plt
import numpy as np
import pytz


from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
import statsmodels.api as sm

from matplotlib.dates import DateFormatter
from matplotlib.ticker import MaxNLocator
from mpl_toolkits.axes_grid1.inset_locator import inset_axes, mark_inset

from matplotlib.dates import HourLocator, DateFormatter

import glob

import matplotlib.gridspec as gridspec
from scipy.stats import linregress

### PHOENIX analysis
last update: April 25, 2025

This is supposed to work for __Bokeh__. 

Create a script that:

    - reads the data 
    - cleans it up 
    - 10min running median
    - flags for RH 
    - categorize data based on EPA limits
    - plots PM2.5 and PM10 in bokeh

### read the csv files that Nikos's code downloads
- get dataframes for each csv

In [2]:
data_path = 'data'
df_info = pd.read_csv("PHOENIX_info.csv")


for filename in os.listdir(data_path):
    if filename.lower().endswith(".csv"):
        file_path = os.path.join(data_path, filename)
        df_name = os.path.splitext(filename)[0].replace("-", "_").lower()  # sanitize and lowercase
        globals()[df_name] = pd.read_csv(file_path)
        
for _, row in df_info.iterrows():
    phoenix_id = row['PHOENIX_ID']
    
    if pd.isna(phoenix_id):
        continue  # skip rows with missing ID
    
    df_name = phoenix_id.replace("-", "_").lower()

    if df_name in globals():
        df = globals()[df_name]

        # Check if "device name" in df matches phoenix_id
        if 'device_name' in df.columns and phoenix_id in df['device_name'].values:
            # Add all columns from row to each matching row in the df
            for col in df_info.columns:
                if col != 'PHOENIX_ID':
                    df[col] = row[col]

            globals()[df_name] = df  # reassign in case you're using the updated version later

#### Now I want to clean up the data. Basically get rid of some columns and work only with the basics

In [3]:
# --- 1. Setup and load raw data ---
columns_to_keep = [
    'timestamp_local', 'met.rh', 'met.temp', 
    'pm1', 'pm25', 'pm10', 'device_name',
    'DATETIME installed (mm/dd/yy hh:mm)', 'LAT', 'LON'
]

# Load all CSVs into globals
for filename in os.listdir(data_path):
    if filename.lower().endswith(".csv"):
        file_path = os.path.join(data_path, filename)
        df_name = os.path.splitext(filename)[0].replace("-", "_").lower()
        globals()[df_name] = pd.read_csv(file_path)

#### Start from Datetime installed and let's create 10 min running median

In [4]:
# --- 1. Setup ---
columns_to_keep = [
    'timestamp_local', 'met.rh', 'met.temp', 
    'pm1', 'pm25', 'pm10', 'device_name',
    'DATETIME installed (mm/dd/yy hh:mm)', 'LAT', 'LON'
]

pm_columns = ['pm1', 'pm25', 'pm10']

# --- 2. Load raw CSVs into globals ---
for filename in os.listdir(data_path):
    if filename.lower().endswith(".csv"):
        file_path = os.path.join(data_path, filename)
        df_name = os.path.splitext(filename)[0].replace("-", "_").lower()
        globals()[df_name] = pd.read_csv(file_path)

# --- 3. Clean, filter, and compute rolling medians ---
dfs_clean = []
dfs_rm = []
titles = []

for _, row in df_info.iterrows():
    phoenix_id = row['PHOENIX_ID']
    
    if pd.isna(phoenix_id):
        continue  # skip rows with missing ID

    df_key = phoenix_id.replace("-", "_").lower()
    
    if df_key in globals():
        df_raw = globals()[df_key]

        # Check device name match
        if 'device_name' in df_raw.columns and phoenix_id in df_raw['device_name'].values:
            # Attach metadata
            for col in df_info.columns:
                if col != 'PHOENIX_ID':
                    df_raw[col] = row[col]

            # Keep only relevant columns
            df_clean = df_raw[[col for col in columns_to_keep if col in df_raw.columns]].copy()

            # Parse datetime columns
            if 'timestamp_local' in df_clean.columns:
                df_clean['timestamp_local'] = pd.to_datetime(df_clean['timestamp_local'], errors='coerce')
            if 'DATETIME installed (mm/dd/yy hh:mm)' in df_clean.columns:
                df_clean['DATETIME installed (mm/dd/yy hh:mm)'] = pd.to_datetime(
                    df_clean['DATETIME installed (mm/dd/yy hh:mm)'], errors='coerce'
                )

            # Drop pre-installation data
            if all(c in df_clean.columns for c in ['timestamp_local', 'DATETIME installed (mm/dd/yy hh:mm)']):
                install_time = df_clean['DATETIME installed (mm/dd/yy hh:mm)'].iloc[0]
                df_clean = df_clean[df_clean['timestamp_local'] >= install_time]

            # Save cleaned DataFrame
            clean_name = f"{df_key}_clean"
            globals()[clean_name] = df_clean

            # If valid for plotting, store in list
            if 'pm10' in df_clean.columns and 'timestamp_local' in df_clean.columns:
                dfs_clean.append(df_clean)
                titles.append(phoenix_id)

            # --- Rolling median calculation ---
            df_rm = df_clean.copy()

            if not pd.api.types.is_datetime64_any_dtype(df_rm['timestamp_local']):
                df_rm['timestamp_local'] = pd.to_datetime(df_rm['timestamp_local'], errors='coerce')

            df_rm = df_rm.set_index('timestamp_local').sort_index()

            for col in pm_columns:
                if col in df_rm.columns:
                    df_rm[f"{col}_rm"] = df_rm[col].rolling("10min").median()

            df_rm.reset_index(inplace=True)

            # Save rolling median DataFrame
            rm_name = f"{df_key}_rm"
            globals()[rm_name] = df_rm
            dfs_rm.append(df_rm)


### Flag for high RH 
- Coleen noticed there were issues with high RH and huge spikes. 
- Paul: these need to be cleaned up before we can start sharing data 
- for now, until we get a better to do this let's flag for high RH (>80%)

In [5]:
dfs_flagged = []

for df in dfs_rm:
    df_flagged = df.copy()
    df_flagged['rh_flag'] = (df_flagged['met.rh'] > 80).astype(bool)
    dfs_flagged.append(df_flagged)

dfs_no_rh = [df[df['rh_flag'] == False].copy() for df in dfs_flagged]

In [6]:
for i in range(len(dfs_no_rh)):
    dfs_no_rh[i]['timestamp_local'] = pd.to_datetime(dfs_no_rh[i]['timestamp_local'])
    dfs_no_rh[i] = dfs_no_rh[i].drop_duplicates(subset='timestamp_local')

### Get rid of unwanted columns for now
- create two dfs, one for PM2.5 and one for PM10

In [43]:


df_all = pd.concat(dfs_no_rh, ignore_index=True)
df_all = df_all.set_index('timestamp_local')

df_avg = (
    df_all
    .groupby('device_name')
    .resample('10min')
    .agg({
        'pm25_rm': 'mean',
        'pm10_rm': 'mean',
        'LAT': 'first',
        'LON': 'first'
    })
    .reset_index()
)

columns_to_drop = ["pm10_rm"]
df_pm25 = df_avg.drop(columns=columns_to_drop)

columns_to_drop = ["pm25_rm"]
df_pm10 = df_avg.drop(columns=columns_to_drop)

def categorize_pm10(val):
    if val <= 54:
        return 'Good'
    elif val <= 154:
        return 'Moderate'
    elif val <= 254:
        return 'Unhealthy for Sensitive Groups'
    elif val <= 354:
        return 'Unhealthy'
    elif val <= 424:
        return 'Very Unhealthy'
    else:
        return 'Hazardous'

def categorize_pm25(val):
    if val <= 9:
        return 'Good'
    elif val <= 35.4:
        return 'Moderate'
    elif val <= 55.4:
        return 'Unhealthy for Sensitive Groups'
    elif val <= 125.4:
        return 'Unhealthy'
    elif val <= 225.4:
        return 'Very Unhealthy'
    else:
        return 'Hazardous'

df_pm10['epa_category'] = df_pm10['pm10_rm'].apply(categorize_pm10)
df_pm25['epa_category'] = df_pm25['pm25_rm'].apply(categorize_pm25)

### Now that we have clean data let's try and get Bokeh to work

In [44]:
from bokeh.models import ColumnDataSource, HoverTool, CategoricalColorMapper
from bokeh.plotting import figure, show
from pyproj import Transformer
import panel as pn
pn.extension()

# --- Step 1: Define EPA colors (both PM10 and PM2.5) ---
epa_colors = {
    'Good': '#00E400',
    'Moderate': '#FFFF00',
    'Unhealthy for Sensitive Groups': '#FF7E00',
    'Unhealthy': '#FF0000',
    'Very Unhealthy': '#8F3F97',
    'Hazardous': '#7E0023'
}

# --- Step 2: Convert lat/lon to Web Mercator for both ---
transformer = Transformer.from_crs("epsg:4326", "epsg:3857", always_xy=True)

df_pm10['x'], df_pm10['y'] = transformer.transform(df_pm10['LON'].values, df_pm10['LAT'].values)
df_pm25['x'], df_pm25['y'] = transformer.transform(df_pm25['LON'].values, df_pm25['LAT'].values)

print(df_pm10)

# --- Step 3: Get latest timestamp for each ---
latest_time_pm10 = df_pm10['timestamp_local'].max()
latest_time_pm25 = df_pm25['timestamp_local'].max()

df_pm10_latest = df_pm10[df_pm10['timestamp_local'] == latest_time_pm10]
df_pm25_latest = df_pm25[df_pm25['timestamp_local'] == latest_time_pm25]

# --- Step 4: Create sources ---
source_pm10 = ColumnDataSource(df_pm10_latest)
source_pm25 = ColumnDataSource(df_pm25_latest)

# --- Step 5: Color mappers ---
color_mapper_pm10 = CategoricalColorMapper(factors=list(epa_colors.keys()), palette=list(epa_colors.values()))
color_mapper_pm25 = CategoricalColorMapper(factors=list(epa_colors.keys()), palette=list(epa_colors.values()))

# --- Step 6: Set up figure ---
p = figure(x_axis_type="mercator", y_axis_type="mercator",
           width=900, height=700, title=f"PM10 (Hexagons) + PM2.5 (Circles) — {latest_time_pm10:%Y-%m-%d %H:%M}")

p.add_tile("CartoDB.Positron")

# --- Step 7: Draw PM10 hexagons ---
p.scatter(x='x', y='y', size=30, source=source_pm10,
          marker='hex',
          color={'field': 'epa_category', 'transform': color_mapper_pm10},
          fill_alpha=1, line_color='black', line_width=1.5, legend_label="PM10 (hexagons)")

# --- Step 8: Draw PM2.5 circles on top ---
p.scatter(x='x', y='y', size=20, source=source_pm25,
          marker='circle',
          color={'field': 'epa_category', 'transform': color_mapper_pm25},
          fill_alpha=1, line_color='black', line_width=1.5, legend_label="PM2.5 (circles)")

# --- Step 9: Hover tool ---
hover = HoverTool(tooltips=[
    ("Device", "@device_name"),
    ("PM Value", "@pm10_rm{0.0} or @pm25_rm{0.0}"),
    ("Category", "@epa_category"),
    ("Time", "@timestamp_local{%F %T}")
], formatters={'@timestamp_local': 'datetime'})

p.add_tools(hover)
p.legend.location = "top_left"
p.legend.click_policy = "hide"

pn.pane.Bokeh(p).servable()


                 device_name     timestamp_local    pm10_rm       LAT  \
0                 PHOENIX-01 2025-02-23 10:00:00   3.238000  34.17978   
1                 PHOENIX-01 2025-02-23 10:10:00   5.109700  34.17978   
2                 PHOENIX-01 2025-02-23 10:20:00        NaN       NaN   
3                 PHOENIX-01 2025-02-23 10:30:00        NaN       NaN   
4                 PHOENIX-01 2025-02-23 10:40:00   5.340750  34.17978   
...                      ...                 ...        ...       ...   
191702  PHOENIX_HabreLab_588 2025-04-28 10:40:00  20.263000  34.18340   
191703  PHOENIX_HabreLab_588 2025-04-28 10:50:00  19.363500  34.18340   
191704  PHOENIX_HabreLab_588 2025-04-28 11:00:00  21.743000  34.18340   
191705  PHOENIX_HabreLab_588 2025-04-28 11:10:00  23.965000  34.18340   
191706  PHOENIX_HabreLab_588 2025-04-28 11:20:00  19.787857  34.18340   

              LON epa_category             x             y  
0      -118.11506         Good -1.314851e+07  4.052968e+06  
1

Bokeh(figure)

In [55]:
from bokeh.models import ColumnDataSource, HoverTool, CategoricalColorMapper, DatetimeRangeSlider, CustomJS
from bokeh.plotting import figure, output_file, save
from bokeh.layouts import column
from pyproj import Transformer


# --- Step 1: Define EPA colors (both PM10 and PM2.5) ---
epa_colors = {
    'Good': '#00E400',
    'Moderate': '#FFFF00',
    'Unhealthy for Sensitive Groups': '#FF7E00',
    'Unhealthy': '#FF0000',
    'Very Unhealthy': '#8F3F97',
    'Hazardous': '#7E0023'
}

# --- Step 2: Convert lat/lon to Web Mercator for both ---
transformer = Transformer.from_crs("epsg:4326", "epsg:3857", always_xy=True)
df_pm10['x'], df_pm10['y'] = transformer.transform(df_pm10['LON'].values, df_pm10['LAT'].values)
df_pm25['x'], df_pm25['y'] = transformer.transform(df_pm25['LON'].values, df_pm25['LAT'].values)

# --- Step 3: Get unique timestamps ---
# Use string representations of timestamps to avoid type comparison issues
df_pm10['timestamp_str'] = df_pm10['timestamp_local'].dt.strftime('%Y-%m-%d %H:%M:%S')
df_pm25['timestamp_str'] = df_pm25['timestamp_local'].dt.strftime('%Y-%m-%d %H:%M:%S')

# Get unique timestamp strings
unique_timestamp_strs = sorted(pd.unique(pd.concat([df_pm10['timestamp_str'], df_pm25['timestamp_str']])))

# --- Step 4: Create date/time controls ---
# Create a time selector with formatted display
#time_selector = pn.widgets.Select(
#    name='Time',
#    options=[(ts, ts) for ts in unique_timestamp_strs],  # Using the same string for value and label
#    value=unique_timestamp_strs[-1]  # Start with most recent
#)

start_str = '2025-04-08'
end_str = '2025-04-09'
    
# Filter timestamps where the date part is within range
filtered_timestamps = [
    ts for ts in unique_timestamp_strs 
    if start_str <= ts.split()[0] <= end_str
]

df_pm25 = df_pm25[df_pm25['timestamp_str'].isin(filtered_timestamps)]
#print(df_pm25)


# --- Step 4: Create date range filter ---
# Extract dates from the timestamp strings
dates = sorted(set(ts.split()[0] for ts in unique_timestamp_strs))
start_date = df_pm25['timestamp_str'].min()
#dt.datetime.strptime(dates[0], '%Y-%m-%d').date()
end_date = df_pm25['timestamp_str'].max()
#dt.datetime.strptime(dates[-1], '%Y-%m-%d').date()


latest_time_pm25 = df_pm25['timestamp_str'].max()
init_data = df_pm25[df_pm25['timestamp_str'] == latest_time_pm25]

source_visible = ColumnDataSource(init_data)

# Set up figure - using the formatted time string for the title
plot = figure(x_axis_type="mercator", y_axis_type="mercator",
               width=900, height=700, title=f"PM10 (Hexagons) + PM2.5 (Circles) ")
plot.add_tile("CartoDB.Positron")

# Color mappers
color_mapper_pm10 = CategoricalColorMapper(factors=list(epa_colors.keys()), palette=list(epa_colors.values()))
color_mapper_pm25 = CategoricalColorMapper(factors=list(epa_colors.keys()), palette=list(epa_colors.values()))
    

    
# Draw PM10 hexagons

#p.scatter(x='x', y='y', size=30, source=source_visible,
#              marker='hex',
#              color={'field': 'epa_category', 'transform': color_mapper_pm25},
 #             fill_alpha=1, line_color='black', line_width=1.5, legend_label="PM25 (hexagons)")

dslider = DateSlider(
    name='Date',
    start=start_date,
    end=end_date,
    value=end_date
)

callback = CustomJS(args=dict(source_pm25=source_pm25,source_visible=source_visible,dslider=dslider)
                      ,code='''
                      var date = dslider.value;
                      console.log(date)
                      //var matched_timestamp = source_pm25.data.filter(function(x) {return x.timestamp_local == date});
                      var point_visible = source_visible.data;
                      var point_available = source_pm25.data;
                      
                      //point_visible.epa_category.push(matched_timestamp.epa_category);
                      //console.log(matched_timestamp)
                      
                      point_visible.x = []
                      point_visible.y = []
                      point_visible.epa_category = []

                      // Update the visible data
                      for(var i = 0; i < point_available.x.length; i++) {
                          if (point_available.timestamp_local == date){
                            point_visible.x.push(point_available.x[i]);
                            point_visible.y.push(point_available.y[i]);
                            point_visible.epa_category.push(point_available.epa_category[i]);
                        }   
                      }
                      source_visible.change.emit();
                      ''')

# --- Step 5: Create timestamp filtering function ---
dslider.js_on_change('value', callback)



# Hover tool
#hover = HoverTool(tooltips=[
#        ("Device", "@device_name"),
#        ("PM Value", "@pm10_rm{0.0} or @pm25_rm{0.0}"),
#        ("Category", "@epa_category"),
#        ("Time", "@timestamp_local{%F %T}")
#    ], formatters={'@timestamp_local': 'datetime'})
#p.add_tools(hover)
#p.legend.location = "top_left"
#p.legend.click_policy = "hide"

show(plot)
#layout = column(p, dslider)
# output_notebook()
#show(layout)
#output_file('output.html')
#save(layout)



RuntimeError: Models must be owned by only a single document, UnionRenderers(id='001dc6f0-cddf-4b59-8aff-df7ae715cd0b', ...) is already in a doc

In [42]:
import numpy as np

from bokeh.layouts import column, row
from bokeh.models import ColumnDataSource, CustomJS, Slider
from bokeh.plotting import figure, show

x = np.linspace(0, 10, 500)
y = np.sin(x)

source = ColumnDataSource(data=dict(x=x, y=y))

plot = figure(y_range=(-10, 10), width=400, height=400)

plot.line('x', 'y', source=source, line_width=3, line_alpha=0.6)

amp = DateSlider(start=start_date,
    end=end_date,
    value=end_date, title="Amplitude")
freq = Slider(start=0.1, end=10, value=1, step=.1, title="Frequency")
phase = Slider(start=-6.4, end=6.4, value=0, step=.1, title="Phase")
offset = Slider(start=-9, end=9, value=0, step=.1, title="Offset")

callback = CustomJS(args=dict(source=source, amp=amp, freq=freq, phase=phase, offset=offset),
                    code="""
    const A = amp.value
    const k = freq.value
    const phi = phase.value
    const B = offset.value

    const x = source.data.x
    const y = Array.from(x, (x) => B + A*Math.sin(k*x+phi))
    source.data = { x, y }
""")

amp.js_on_change('value', callback)
freq.js_on_change('value', callback)
phase.js_on_change('value', callback)
offset.js_on_change('value', callback)

show(row(plot, column(amp, freq, phase, offset)))